# Sphere function optimization (using the Island Model).

The general equation is given by:

$f(\mathbf{x}) = \sum_{i=1}^{M} x^2_i$,  with  $-\infty \le x_i \le +\infty$,
and global minimum found at: $f(x_1, x_2, ..., x_M) = f(0, 0, ..., 0) = 0$.

### First we import python libraries and set up the directory of our code.

In [3]:
import os, sys
import numpy as np
from math import fsum
import matplotlib.pyplot as plt

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [5]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.island_model_ga import IslandModelGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

# Import Migration Operator(s).
from pygenalgo.operators.migration.clockwise_migration import ClockwiseMigration

### Define the sphere function, which plays also the role of the 'fitness' function.

In addition, we define the '_func' which takes the role of the 'random()' method of the Genes. Every time we
want to 'mutate' a gene this function will be called that returns 'valid', but random values for the gene.

In [7]:
# Sphere function.
def fun_Sphere(individual: Chromosome, f_min: bool = True):
    
    # Compute the shpere function.
    f_val = fsum([xi.datum**2 for xi in individual])
    
    # Return the negative (to account for minimization).
    return -f_val if f_min else f_val
# _end_def_

# Random function ~N(0,1).
# Since 'xi' are allowed to take any real value in [-inf, +inf], by setting this
# to the Normal(0, 1) essentially we can get both negative and positive values.
_func = lambda: np.random.normal()

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by '_func'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [9]:
# Define the number of genes.
# In this problem the number of genes can be any positive integer [1, M]
M = 10

# Define the number of chromosomes.
N = 400

# Initial population.
# It is important to note that the initial population is randomly generated with valid values.
population = [Chromosome([Gene(np.random.uniform(-10.0, +10.0), _func) for j in range(M)], np.nan, True)
              for i in range(N)]

# Create the IslandModelGA object that will carry on the optimization.
toy_GA = IslandModelGA(initial_pop=population,
                       fit_func=fun_Sphere,
                       num_islands=5,
                       select_op=LinearRankSelector(),
                       mutate_op=RandomMutator(),
                       cross_op=UniformCrossover(),
                       migrate_op=ClockwiseMigration())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [11]:
toy_GA(epochs=2500, elitism=True, f_tol=1.0e-6, allow_migration=True, n_periods=10, verbose=True)


Current period 1 / 10:

Best Fitness in island 0 is:= -0.00081
Best Fitness in island 1 is:= -0.00228
Best Fitness in island 2 is:= -0.00056
Best Fitness in island 3 is:= -0.00276
Best Fitness in island 4 is:= -0.00086

Current period 2 / 10:

Best Fitness in island 0 is:= -0.00014
Best Fitness in island 1 is:= -0.00022
Best Fitness in island 2 is:= -0.00010
Best Fitness in island 3 is:= -0.00004
Best Fitness in island 4 is:= -0.00025

Current period 3 / 10:

Best Fitness in island 0 is:= -0.00006
Best Fitness in island 1 is:= -0.00004
Best Fitness in island 2 is:= -0.00004
Best Fitness in island 3 is:= -0.00002
Best Fitness in island 4 is:= -0.00002

Current period 4 / 10:

Best Fitness in island 0 is:= -0.00001
Best Fitness in island 1 is:= -0.00003
Best Fitness in island 2 is:= -0.00003
Best Fitness in island 3 is:= -0.00002
Best Fitness in island 4 is:= -0.00001

Current period 5 / 10:

Best Fitness in island 0 is:= -0.00001
Best Fitness in island 1 is:= -0.00001
Best Fitness in i

In [12]:
# Extract the optimal solution from the GA.
optimal_solution = toy_GA.best_chromosome()

# Display the (final) optimal value.
print(f"Minimum Found: {fun_Sphere(optimal_solution, f_min=False):.5f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.genome):
    print(f"x{i} = {xi.datum:>10.6f}")
# _end_for_

# True minimum: f(0.0, 0.0, ..., 0.0) = 0.0

Minimum Found: 0.00000

x0 =  -0.000139
x1 =  -0.000150
x2 =   0.000018
x3 =  -0.000130
x4 =   0.000004
x5 =  -0.000305
x6 =  -0.000065
x7 =   0.000069
x8 =   0.000179
x9 =   0.000009


### End of file